In [439]:
# installations

In [440]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [441]:
cd /content/drive/MyDrive/NLP_assignment/assignment04

/content/drive/.shortcut-targets-by-id/1ap4ma5tLjADefUa9-T4Mv9PZ7QdvVjvF/assignment04


In [442]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import itertools

## EXTEND BINGLIU 

In [443]:
filename = '/content/drive/MyDrive/NLP_assignment/assignment04/data/english-hindi-dictionary.txt'

In [444]:
data_df = pd.read_table(filename, names=('en','sep','hi'), delim_whitespace=True)
data_df = data_df.drop(columns=['sep'])

In [445]:
data_dict = data_df.set_index('en').T.to_dict('list')

#display dictionary
data_dict

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


{'the': ['द'],
 '.': ['।'],
 ',': [','],
 'i': ['मैं'],
 'to': ['करने'],
 'and': ['और'],
 'a': ['एक'],
 'it': ['यह'],
 'is': ['है'],
 'of': ['की'],
 'this': ['यह'],
 'for': ['के'],
 'that': ['वह'],
 'in': ['में'],
 'you': ['आप'],
 'with': ['के'],
 'my': ['मेरा'],
 'on': ['पर'],
 'have': ['पास'],
 'but': ['लेकिन'],
 'not': ['नहीं'],
 'was': ['था'],
 '&apos;t': ['&apos;t'],
 ')': [')'],
 'as': ['के'],
 '(': ['('],
 '!': ['!'],
 '&apos;s': ['&apos;s'],
 'are': ['हैं'],
 'so': ['तो'],
 'be': ['होना'],
 'can': ['कर'],
 'one': ['एक'],
 'if': ['अगर'],
 'very': ['बहुत'],
 'they': ['वे'],
 'at': ['पर'],
 '&quot;': ['&quot;'],
 'or': ['या'],
 'great': ['महान'],
 'all': ['सभी'],
 'use': ['उपयोग'],
 'from': ['से'],
 'had': ['था'],
 'good': ['अच्छा'],
 'just': ['बस'],
 'when': ['जब'],
 'an': ['एक'],
 'up': ['ऊपर'],
 'out': ['बाहर'],
 'your': ['अपने'],
 'would': ['होगा'],
 'has': ['है'],
 'no': ['कोई'],
 'get': ['पाएं'],
 'will': ['करेंगे'],
 'like': ['की'],
 'me': ['मुझे'],
 'about': ['के'],
 'came

In [446]:
data_dict['list']

['सूची']

In [447]:
eng_hind_dict_set = set()
for key in data_dict:
  pair = (key, data_dict[key][0])
  eng_hind_dict_set.add(pair)

In [448]:
bingliu = pd.read_csv('/content/drive/MyDrive/NLP_assignment/assignment04/data/BingLiu.csv', sep='\t', header=None)
bingliu.head()

,0,1
0,a+,positive
1,abound,positive
2,abounds,positive
3,abundance,positive
4,abundant,positive


In [449]:
bingliu=bingliu.rename(columns={0: "eng", 1: "lexicon"})
bingliu.head()

,eng,lexicon
0,a+,positive
1,abound,positive
2,abounds,positive
3,abundance,positive
4,abundant,positive


In [450]:
def add_hindi_word(row, data_dict=data_dict):
  eng = row['eng']
  if eng in data_dict: 
      hi = data_dict[eng][0]
  else: 
      hi = None  
  return hi 

In [451]:
bingliu['hindi'] = bingliu.apply(add_hindi_word, axis=1)

In [452]:
bingliu.head()

,eng,lexicon,hindi
0,a+,positive,None
1,abound,positive,abound
2,abounds,positive,abounds
3,abundance,positive,abundance
4,abundant,positive,हुस्न


In [453]:
bingliu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6789 entries, 0 to 6788
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   eng      6789 non-null   object
 1   lexicon  6789 non-null   object
 2   hindi    5013 non-null   object
dtypes: object(3)
memory usage: 159.2+ KB


In [454]:
bingliu = bingliu.dropna()  #drop rows with no hindi translation 

In [455]:
bingliu = bingliu[bingliu['eng'] != bingliu['hindi']]  #keep rows where en != hi

In [456]:
bingliu.head()

,eng,lexicon,hindi
4,abundant,positive,हुस्न
6,accessible,positive,सुलभ
14,accomplish,positive,पूरा
15,accomplished,positive,पूरा
16,accomplishment,positive,उपलब्धि


In [457]:
#bingliu.to_csv('/content/drive/MyDrive/NLP_assignment/assignment04/data/L1.csv', header=True)

DONE!

## WORD2VEC MODEL

In [458]:
eng_w2v_model=Word2Vec.load("/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/400_stopwords_word2vec_eng.model")
# eng_w2v_model=Word2Vec.load("/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/50_word2vec_eng.model")

# eng_w2v_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/NLP_assignment/assignment04/pre-trained-models/GoogleNews-vectors-negative300.bin.gz", binary = True, limit = 100000) 

In [459]:
def english_word2vec(word):
  # returns 5 word vectors based on our corpus
  try:
    result = eng_w2v_model.most_similar(word, topn = 5)
  except:
    result = [[0,0]]
  result.append([word, 0])
  return [word[0] for word in result]

In [460]:
hindi_w2v_model=Word2Vec.load("/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/400_stopwords_word2vec_hindi.model")
# hindi_w2v_model=Word2Vec.load("/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/50_word2vec_hindi.model")

# hindi_w2v_model = Word2Vec.load("/content/drive/MyDrive/NLP_assignment/assignment04/pre-trained-models/hi.bin") 

In [461]:
def hindi_word2vec(word):
  # returns 5 word vectors based on our corpus
  try:
    result = hindi_w2v_model.most_similar(word, topn = 5)
  except:
    result = [[0,0]]
  result.append([word, 0])
  return [word[0] for word in result]

In [462]:
from gensim.models.keyedvectors import KeyedVectors

In [463]:
import numpy as np
from gensim.scripts.glove2word2vec import glove2word2vec
_ = glove2word2vec("/content/drive/MyDrive/NLP_assignment/assignment04/vectors-english-300.txt", "/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/300_glove_gensim_model_eng.model")
eng_glove_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/300_glove_gensim_model_eng.model", binary = False)

_ = glove2word2vec(glove_input_file="/content/drive/MyDrive/NLP_assignment/assignment04/vectors-hindi-300.txt", word2vec_output_file="/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/300_glove_gensim_model_hindi.model")
hindi_glove_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/NLP_assignment/assignment04/trained-models/300_glove_gensim_model_hindi.model", binary=False)


In [464]:
def english_glove(word):
  try:
    result = eng_glove_model.most_similar(word, topn = 5)
  except:
    result = [[0,0]]
  result.append([word, 0])
  return [word[0] for word in result]

In [465]:
def hindi_glove(word):
  try:
    result = hindi_glove_model.most_similar(word, topn = 5)
  except:
    result = [[0,0]]
  result.append([word, 0])
  return [word[0] for word in result]

In [466]:
bingliu_extended_dict = dict()
for ind in bingliu.index:
  bingliu_extended_dict[(bingliu["eng"][ind], bingliu["hindi"][ind])] = bingliu["lexicon"][ind] 

In [467]:
def extend_embeddings():
  extensions_added = True
  number_of_additions = 0
  bingliu_extensions = dict()
  while extensions_added:
    extensions_added = False
    for ind in bingliu.index: 
     eng, hindi = bingliu['eng'][ind], bingliu['hindi'][ind]
     # word2vec
     eng_neighbours = english_word2vec(eng)
     hindi_neighbours = hindi_word2vec(hindi)
     if eng_neighbours == 0 or hindi_neighbours == 0:
       continue
     pairs = itertools.product(eng_neighbours,hindi_neighbours)
     for pair in pairs:
       eng_word, hindi_word = pair[0], pair[1]
       if (eng_word, hindi_word) in eng_hind_dict_set and (eng_word, hindi_word) not in bingliu_extended_dict:
         print("adding", (eng_word, hindi_word), bingliu["lexicon"][ind])
         extensions_added = True
         bingliu_extended_dict[(eng_word, hindi_word)] = bingliu["lexicon"][ind]
         bingliu_extensions[(eng_word, hindi_word)] = bingliu["lexicon"][ind]
         number_of_additions += 1
  return number_of_additions, bingliu_extensions

      

In [468]:
addition_number, addition_entries = extend_embeddings()
print(addition_number)
addition_entries

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


adding ('even', 'यहां') positive
adding ('pro', 'प्रो') positive
adding ('either', 'या') positive
adding ('much', 'ज्यादा') positive
adding ('although', 'हालांकि') negative
adding ('place', 'जगह') negative
adding ('one', 'एक') negative
7


{('although', 'हालांकि'): 'negative',
 ('either', 'या'): 'positive',
 ('even', 'यहां'): 'positive',
 ('much', 'ज्यादा'): 'positive',
 ('one', 'एक'): 'negative',
 ('place', 'जगह'): 'negative',
 ('pro', 'प्रो'): 'positive'}

In [469]:
def extend_embeddings_glove():
  extensions_added = True
  number_of_additions = 0
  bingliu_extensions = dict()
  while extensions_added:
    extensions_added = False
    for ind in bingliu.index: 
     eng, hindi = bingliu['eng'][ind], bingliu['hindi'][ind]
     # glove
     eng_neighbours = english_glove(eng)
     hindi_neighbours = hindi_glove(hindi)
     glove_pairs = []
     if eng_neighbours != 0 and hindi_neighbours != 0:
       glove_pairs = itertools.product(eng_neighbours,hindi_neighbours)
     for pair in glove_pairs:
       eng_word, hindi_word = pair[0], pair[1]
       if (eng_word, hindi_word) in eng_hind_dict_set and (eng_word, hindi_word) not in bingliu_extended_dict:
         print("adding", (eng_word, hindi_word), bingliu["lexicon"][ind])
         extensions_added = True
         bingliu_extended_dict[(eng_word, hindi_word)] = bingliu["lexicon"][ind]
         bingliu_extensions[(eng_word, hindi_word)] = bingliu["lexicon"][ind]
         number_of_additions += 1
  return number_of_additions, bingliu_extensions

In [470]:
addition_number, addition_entries = extend_embeddings_glove()
print(addition_number)
addition_entries

adding ('tasty', 'स्वादिष्ट') positive
adding ('makes', 'सबके') positive
adding ('sized', 'हल्का') positive


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


3


{('makes', 'सबके'): 'positive',
 ('sized', 'हल्का'): 'positive',
 ('tasty', 'स्वादिष्ट'): 'positive'}

In [474]:
print(bingliu_extended_dict)

{('abundant', 'हुस्न'): 'positive', ('accessible', 'सुलभ'): 'positive', ('accomplish', 'पूरा'): 'positive', ('accomplished', 'पूरा'): 'positive', ('accomplishment', 'उपलब्धि'): 'positive', ('accomplishments', 'उपलब्धियों'): 'positive', ('accurate', 'सटीक'): 'positive', ('accurately', 'यथासंभव'): 'positive', ('achievement', 'उपलब्धि'): 'positive', ('achievements', 'उपलब्धियाँ'): 'positive', ('acumen', 'कौशल'): 'positive', ('adjustable', 'समायोजनीय'): 'positive', ('admirable', 'प्रशंसनीय'): 'positive', ('admire', 'प्रशंसा'): 'positive', ('admirer', 'प्रशंसक'): 'positive', ('adorable', 'प्यारे'): 'positive', ('adore', 'प्यार'): 'positive', ('advanced', 'उन्नत'): 'positive', ('advantage', 'लाभ'): 'positive', ('advantages', 'लाभ'): 'positive', ('advocates', 'हिन्दुत्व'): 'positive', ('affection', 'स्नेह'): 'positive', ('affinity', 'संबंध'): 'positive', ('affirmative', 'सकारात्मक'): 'positive', ('afford', 'बर्दाश्त'): 'positive', ('affordable', 'किफायती'): 'positive', ('agile', 'एजाइल'): 'po

In [476]:
final_dict = {
    "eng" : [],
    "hindi" : [],
    "lexicon" : []
}
for key in bingliu_extended_dict:
  final_dict["eng"].append(key[0])
  final_dict["hindi"].append(key[1])
  final_dict["lexicon"].append(bingliu_extended_dict[key])

final_df = pd.DataFrame(final_dict)
final_df.to_csv("extended-lexicon.csv", index = True, header = True)


In [477]:
final_df

,eng,hindi,lexicon
0,abundant,हुस्न,positive
1,accessible,सुलभ,positive
2,accomplish,पूरा,positive
3,accomplished,पूरा,positive
4,accomplishment,उपलब्धि,positive
...,...,...,...
2118,place,जगह,negative
2119,one,एक,negative
2120,tasty,स्वादिष्ट,positive
2121,makes,सबके,positive
